In [ ]:
import requests
import re
from tqdm import tqdm
import time
import pandas as pd

In [ ]:
METACYC_PROTEINS_PATH = '/Users/Itai/Library/Mobile Documents/com~apple~CloudDocs/from_box/Grad/research/molecule_databases/Metacyc_v26.5/data/protein-seq-ids-unreduced.dat'

In [ ]:
def query_genbank_proteins (id_list):
    """
    Retrieve protein FASTA for list of GenBank accession IDs
    """
    
    db = 'protein'
    ids_string = ','.join(id_list)
    base = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/'
    
    url = base + f"epost.fcgi?db={db}&id={ids_string}"
    
    res = requests.get(url).text
    web = re.findall('(?<=<WebEnv>).*(?=<\/WebEnv>)', res)
    key = re.findall('(?<=<QueryKey>).*(?=<\/QueryKey>)', res)
    if len(web) and len(key):
        web = web[0]
        key = key[0] 
        url2 = base + f"efetch.fcgi?db={db}&query_key={key}&WebEnv={web}"+ "&rettype=fasta&retmode=text"
        res = requests.get(url2).text.split('\n\n>')
    else:
        res = None
    return res

def query_batched (full_id_list, batch_size=100):
    """
    Retrieve protein FASTA for list of GenBank accession IDs in batches
    """
    list_of_lists = [full_id_list[i:i + batch_size] for i in range(0, len(full_id_list), batch_size)]
    query_results = []
    for ls in tqdm(list_of_lists):
        res = query_genbank_proteins(ls)
        query_results.extend(res)
    return query_results
    
def parse_query_results (result):
    """
    Extract information from queries to GenBank 
    """
    header = result.split('\n')[0]

    aid = header.split(' ')[0].replace('>','')
    escaped_aid = re.escape(aid)
    if '[' in header:
        species = re.findall('(?<=\[).*(?=\])', header)[0]
        common_name = re.findall(f'(?<={escaped_aid} ).*(?= \[)', header)[0]
    else:
        species = 'NA'
        print (aid)
        try:
            common_name = re.findall(f'(?<={escaped_aid} ).*$', header)[0]
        except:
            print (header)
            common_name = None

    sequence = ''.join(result.split('\n')[1:])
    
    return {'id':aid, 'species':species, 'common_name':common_name, 'sequence':sequence}

## GenBank

In [ ]:
genbank_accessions = []
with open(METACYC_PROTEINS_PATH, 'r') as f:
    for line in f:
        genbank_accessions.extend(re.findall('(?<=PID:).+?(?=\")', line))

In [ ]:
queried = query_batched(genbank_accessions)

In [ ]:
queried

In [ ]:
parsed_genbank_results = [parse_query_results(q) for q in tqdm(queried)]

In [ ]:
ids = [p['id'].split('.')[0] for p in parsed_genbank_results]

In [ ]:
missing = [g for g in genbank_accessions if g.split('.')[0] not in ids]
print (len(set(missing)))
print (set(missing))

In [ ]:
gid_dict = {}
for gid in tqdm(set(missing)):
    res = query_genbank_proteins([gid])
    if res is not None:
        gid_dict[gid] = res[0][1:].split(' ')[0]
        time.sleep(0.5)
    else:
        print (gid)

In [ ]:
converted_genbank_ids = [g if g not in gid_dict.keys() else gid_dict[g] for g in genbank_accessions]

In [ ]:
new_missing = [g for g in converted_genbank_ids if g.split('.')[0] not in ids and g not in ids]
print (new_missing)
print (len(new_missing))

In [ ]:
missing_queried = query_batched(new_missing, batch_size=1)

In [ ]:
parsed_genbank_results.extend([parse_query_results(q) for q in tqdm(missing_queried)])

In [ ]:
new_ids = [p['id'].split('.')[0] for p in parsed_genbank_results]

In [ ]:
final_missing = [g for g in converted_genbank_ids if g.split('.')[0] not in new_ids and g not in new_ids]
final_missing

In [ ]:
gid_dict

In [ ]:
pid_to_sequence = {}

for res in parsed_genbank_results:
    if 'id' in res.keys():
        pid_to_sequence[res['id'].split('.')[0]] = res['sequence']
        
for gid in gid_dict:
    if gid_dict[gid].split('.')[0] in pid_to_sequence.keys():
        pid_to_sequence[gid] = pid_to_sequence[gid_dict[gid].split('.')[0]]

## Uniprot

In [ ]:
uniprot_accessions = []
with open(METACYC_PROTEINS_PATH, 'r') as f:
    for line in f:
        uniprot_accessions.extend(re.findall('(?<=UNIPROT:).*?\"', line))

In [ ]:
unaccounted_for_accessions = [u for u in uniprot_accessions if ':' in u and u.split(':')[1] not in metacyc_proteins_df['REACTION_ID']]
print (len(unaccounted_for_accessions))

In [ ]:
####
#### Used .py script instead
####

In [ ]:
# from Uniprot
# >db|UniqueIdentifier|EntryName ProteinName OS=OrganismName OX=OrganismIdentifier [GN=GeneName ]PE=ProteinExistence SV=SequenceVersion
# db is 'sp' for UniProtKB/Swiss-Prot and 'tr' for UniProtKB/TrEMBL.
# UniqueIdentifier is the primary accession number of the UniProtKB entry.
# EntryName is the entry name of the UniProtKB entry.
# ProteinName is the recommended name of the UniProtKB entry as annotated in the RecName field. For UniProtKB/TrEMBL entries without a RecName field, the SubName field is used. In case of multiple SubNames, the first one is used. The 'precursor' attribute is excluded, 'Fragment' is included with the name if applicable.
# OrganismName is the scientific name of the organism of the UniProtKB entry.
# OrganismIdentifier is the unique identifier of the source organism, assigned by the NCBI.
# GeneName is the first gene name of the UniProtKB entry. If there is no gene name, OrderedLocusName or ORFname, the GN field is not listed.
# ProteinExistence is the numerical value describing the evidence for the existence of the protein.
# SequenceVersion is the version number of the sequence.

def extract_header_information(header):
    regex_pattern = r'>?([^|]+)\|([^|]+)\|([^ ]+?) (.*?)(?: OS=(.+?))(?: OX=(.+))?(?: GN=(.+?))?(?: PE=(.+?))(?: SV=(.+?))'
    match = re.match(regex_pattern, header)
    if match:
        db_value = match.group(1)
        unique_identifier = match.group(2)
        entry_name = match.group(3)
        protein_name = match.group(4)
        organism_name = match.group(5)
        organism_identifier = match.group(6)
        gene_name = match.group(7)
        protein_existence = match.group(8)
        sequence_version = match.group(9)

        return {
            "db": db_value,
            "UniqueIdentifier": unique_identifier,
            "EntryName": entry_name,
            "ProteinName": protein_name,
            "OrganismName": organism_name,
            "OrganismIdentifier": organism_identifier,
            "GeneName": gene_name,
            "ProteinExistence": protein_existence,
            "SequenceVersion": sequence_version
        }
    else:
        print ("Failed parsing")
        return None

# Example header with different 'db' values
# header_example = '>sp|Q01740|FMO1_HUMAN Flavin-containing monooxygenase 1 OS=Homo sapiens OX=9606 GN=FMO1 PE=1 SV=3'
header_example = '>sp|Q01740|FMO1_HUMAN Flavin-containing monooxygenase 1 OS=Homo sapiens OX=9606 PE=1 SV=3'
# For another 'db' value
# header_example = ">db2|789012|AnotherEntry AnotherProtein OS=Mouse OX=10090 PE=2 SV=2"

result = extract_header_information(header_example)
if result:
    print("Extracted Information:")
    for key, value in result.items():
        print(f"{key}: {value}")
else:
    print("Header format doesn't match the pattern.")


def parse_uniprot_result (uniprot_result):
    header = uniprot_result.split('\n')[0]
    sequence = ''.join(uniprot_result.split('\n')[1:])
    info = extract_header_information(header)
    if info is not None:
        info['sequence'] = sequence
    else:
        print (header)
        print (sequence)
    return info

def parse_uniprot_batch (uniprot_batch):
    result_ls = []
    for uniprot_result in uniprot_batch.split('\n>'):
        if uniprot_result is not None:
            parsed = parse_uniprot_result(uniprot_result)
            if parsed is not None:
                result_ls.append(parsed)
    
    return result_ls
# def parse_batched_uniprot_results (batch_results):

In [ ]:
### Load uniprot query results
with open('uniprot_chkpoint.txt', 'r') as f:
    uniprot_returned = f.readlines()

In [ ]:
# parsed_uniprot = []
# for batch in uniprot_returned:
#     parsed_uniprot.extend(parse_uniprot_batch(batch))
parsed_uniprot = parse_uniprot_batch('\n'.join(uniprot_returned))

In [ ]:
len(parsed_uniprot)

In [ ]:
parsed_uniprot[167838]

In [ ]:
#### SAVED JUST IN CASE
# with open('uniprot_chkpoint.txt', 'w') as f:
#     for line in returned_uniprot:
#         f.write(line)

In [ ]:
def parse_metacyc_proteins(file_path):
    # Define the regular expression pattern for extracting entries
    results = []
    entry_pattern = re.compile(r'\((.*?)\)')

    # Open the file and read its content
    with open(file_path, 'r') as file:
        file_content = file.read()

    # Find all matches of the entry pattern in the file content
    entries = re.findall(r'\((.*?)\)', file_content, re.DOTALL)

    # Process each entry
    for entry in entries:
        # Split the entry into its components
        components = entry.split()

        # Extract R_ID and EC_ID
        r_id, ec_id = [c.strip('(') for c in components[:2]]

        # Extract PIDs
        pids = components[2:]

        # Remove quotes from PIDs
        pids = [pid.strip('"') for pid in pids]

        # Print or process the extracted information as needed
        
        for pid in pids:
            if ':' in pid:
                results.append({'REACTION_ID':r_id, 'EC':ec_id, 'SEQUENCE_ID':pid.split(':')[1], 'SEQUENCE_DB':pid.split(':')[0]})
            else:
                print (pid)
        
    return results
        

# Replace 'your_file.txt' with the actual path to your text file
metacyc_proteins_info = parse_metacyc_proteins(METACYC_PROTEINS_PATH)

In [ ]:
for res in parsed_uniprot:
    pid_to_sequence[res['UniqueIdentifier']] = res['sequence']

In [ ]:
cleaned_gid_dict = {}

for k,v in gid_dict.copy().items():
    if 'Error' in v or (v==''):
        pass
    elif '|' in v and len(v.split('|')[1]) > 0:
        cleaned_gid_dict[k] = gid_dict[k].split('|')[1].split('.')[0]
    else:
        cleaned_gid_dict[k] = gid_dict[k].split('.')[0]   

In [ ]:
no_sequence = []
for entry in metacyc_proteins_info:
    clean_id = entry['SEQUENCE_ID'].split('.')[0]
    if clean_id in pid_to_sequence.keys():
        entry['sequence'] = pid_to_sequence[clean_id]
    elif clean_id in gid_dict.keys():
        try:
            entry['sequence'] = pid_to_sequence[cleaned_gid_dict[clean_id]]
        except:
            print ('ERRRORORR', entry)
            no_sequence.append(entry)
    else:
        print (entry)
        no_sequence.append(entry)

In [ ]:
print (len(no_sequence))

## Improve querying for uniprot sequences

In [ ]:
metacyc_proteins_df = pd.DataFrame(metacyc_proteins_info)

In [ ]:
no_sequence_in_df = set(metacyc_proteins_df.drop_duplicates(subset='REACTION_ID')['REACTION_ID'].tolist()) - set(metacyc_proteins_df.dropna(subset=['sequence']).drop_duplicates(subset='REACTION_ID')['REACTION_ID'].tolist())
no_sequence_in_df

In [ ]:
failed_to_get_sequence = metacyc_proteins_df[metacyc_proteins_df['REACTION_ID'].map(lambda x : x in no_sequence_in_df)]['REACTION_ID'].tolist()
metacyc_proteins_df[metacyc_proteins_df['REACTION_ID'].map(lambda x : x in no_sequence_in_df)]

In [ ]:
# metacyc_proteins_df.to_csv('10Nov2023_metacyc_reactions_with_sequences.csv', index=False)

In [ ]:
ids_to_requery = metacyc_proteins_df[metacyc_proteins_df['REACTION_ID'].map(lambda x : x in failed_to_get_sequence)]['SEQUENCE_ID'].tolist()

In [ ]:
ids_to_requery

In [ ]:
requeried_uniprot = []
for u_id in tqdm(ids_to_requery):
    res = requests.get(f"https://rest.uniprot.org/uniprotkb/search?query={u_id}").text
    if "mergeDemergeTo" in res:
        new_id = re.findall('(?<=mergeDemergeTo":\[").+?(?="\])', res)[0]
        res = requests.get(f"https://rest.uniprot.org/uniprotkb/search?query={new_id}&format=fasta").text
    elif '"sequence":{"value":"' in res:
        res = requests.get(f"https://rest.uniprot.org/uniprotkb/search?query={u_id}&format=fasta").text
    requeried_uniprot.append(res)
    time.sleep(0.5)

In [ ]:
parsed_requeried_uniprot = [parse_uniprot_result(r) for r in requeried_uniprot]

In [ ]:
updated_pid_to_sequence = pid_to_sequence.copy()
for idx, entry in zip(ids_to_requery, parsed_requeried_uniprot):
    if entry is not None:
        updated_pid_to_sequence[idx] = entry['sequence']

In [ ]:
for idx in metacyc_proteins_df[metacyc_proteins_df['sequence'].isna()].index:
    if metacyc_proteins_df.loc[idx, 'SEQUENCE_ID'] in updated_pid_to_sequence:
        metacyc_proteins_df.loc[idx, 'sequence'] = updated_pid_to_sequence[metacyc_proteins_df.loc[idx, 'SEQUENCE_ID']]

# metacyc_proteins_df.dropna(subset=['sequence'])

In [ ]:
new_no_sequence_in_df = set(metacyc_proteins_df.drop_duplicates(subset='REACTION_ID')['REACTION_ID'].tolist()) - set(metacyc_proteins_df.dropna(subset=['sequence']).drop_duplicates(subset='REACTION_ID')['REACTION_ID'].tolist())


In [ ]:
new_no_sequence_in_df

In [ ]:
metacyc_proteins_df.dropna(subset=['sequence']).drop_duplicates(subset=['REACTION_ID'])

In [ ]:
metacyc_proteins_df.to_csv('21Nov2023_metacyc_reactions_with_sequences.csv', index=False)

In [ ]:
import json
with open ('protein_id_to_sequence.json', 'w') as f:
    json.dump(updated_pid_to_sequence, f)

In [ ]:
with open('protein_id_to_sequence.json','r') as f:
    test_dict = json.load(f)

In [ ]:
test_dict['AAH16836']

In [ ]:
metacyc_reactions_df = pd.read_csv('metacyc/08Mar2023_metacyc_reaction_smiles_no_cofs.tsv', sep='\t')

In [ ]:
metacyc_reactions_df = metacyc_reactions_df.merge(metacyc_proteins_df.dropna(subset=['sequence']).drop_duplicates('REACTION_ID'), 
                           left_on='UNIQUE-ID', right_on='REACTION_ID', how='left')

In [ ]:
metacyc_reactions_df.to_csv('metacyc/21Nov2023_metacyc_reaction_smiles_no_cofs_with_sequences.tsv', sep='\t', index=False)

## Parse queries from Rhea

In [ ]:
with open('uniprot_chkpoint_for_rhea.txt','r') as f:
    uniprot_rhea_returned = f.readlines()

In [ ]:
parsed_uniprot_rhea = parse_uniprot_batch('\n'.join(uniprot_rhea_returned))

In [ ]:
rhea_proteins_df = pd.read_csv('rhea/rhea2uniprot_sprot.tsv', sep='\t')

In [ ]:
metacyc_proteins_df # pd.DataFrame(metacyc_proteins_info)

In [ ]:
for entry in parsed_uniprot_rhea:
    updated_pid_to_sequence[entry['UniqueIdentifier']] = entry['sequence']

In [ ]:
rhea_proteins_df['sequence'] = [updated_pid_to_sequence[idx] for idx in rhea_proteins_df['ID']]

In [ ]:
rhea_proteins_df.to_csv('12Nov2023_rhea_proteins_with_sequences.csv', index=False)

In [ ]:
rhea_reactions = pd.read_csv('rhea/08Mar2023_rhea_reaction_smiles_no_cofs.csv', sep='\t')

In [ ]:
rid_to_seq = pd.Series(rhea_proteins_df['sequence'].values, index=rhea_proteins_df['RHEA_ID'].values).to_dict()

In [ ]:
idx2seq = {}
for idx in rhea_reactions.index:
    rhea_ids = [int(x) for x in re.findall('(?<=RHEA:)[0-9]+', rhea_reactions.loc[idx,'ID'])]
    for rid in rhea_ids:
        if rid in rid_to_seq.keys():
            idx2seq[idx] = rid_to_seq[rid]
        

In [ ]:
len(idx2seq)

In [ ]:
rhea_reactions['sequence'] = [idx2seq[idx] if idx in idx2seq.keys() else None for idx in rhea_reactions.index]

In [ ]:
rhea_reactions[rhea_reactions['sequence'].isna()]

In [ ]:
rhea_reactions

In [ ]:
rhea_reactions.to_csv('rhea/21Nov2023_rhea_reaction_smiles_no_cofs_with_sequences.csv', index=False, sep='\t')